This notebook is for visualizing the results of the sliding-window QA model approach on the Modern Slavery Dataset. The approach is run in the notebook 'QA-sliding window.ipynb'. The idea behind the approach is to use a pretrained QA model (one trained on SQuAD v2 such that it can return a "no span found" result) to ask questions of the documents in a zero-shot configuration. Since most documents in the dataset are longer than the maximum input length, a sliding window approach is used: after the entire document is tokenized, the QA model is run on successive windows, each slid by stride=128 tokens (~1/4th of the window size). All spans returned by the QA model are recorded in a new dataframe (df_with_segments.parquet). 

This viewer is used to display the spans found by the QA model to investigate whether the model is providing useful filtering. As such, it is important to be able to visualize not only the spans, but also the text that the model ignored. Therefore, the entire documents are printed and the spans that were found are highlighted. The model was run with 6 potentially useful questions. Here, each question is assigned a different color as shown in the legend below. Where multiple questions returned the same span, that span is printed multiple times, once in each color.

In [1]:
# This cell contains all of the code used to generate the highlights. To view the highlights, skip to the results of the 
# following cells.

# Note: the highlighting code doesn't work in all circumstances. This is mostly due to the fact that I didn't record the
# token classes in the dataframe and am having to interpolate the tokens from the strings. In the future, the 
# 'QA-sliding window.ipynb' notebook will be updated to record the token classes in the dataframe and then this visualizer
# can be updated.
import pandas as pd
from transformers import AutoTokenizer

model_name = 'ktrapeznikov/albert-xlarge-v2-squad-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_parquet('df_with_segments.parquet')
questions = [col for col in df.columns if '?' in col]

def print_doc_with_highlights(df, tokenizer, row_id, 
                              questions = [col for col in df.columns if '?' in col]):
    colors = get_colors()
    print_legend(questions, colors)
    
    text = df.loc[row_id,'TEXT']
    all_tokens = tokenizer(text, return_tensors='pt')['input_ids'][:,1:-1] # drop first and last tokens ([CLS] and [SEP])
    token_starts = token_start_locations(text, tokenizer, all_tokens)
    
    spans=find_spans(df, tokenizer, row_id, questions, all_tokens)
    
    if spans.shape[0] == 0:
        # no spans found, so print the whole text as is
        print(text)
    else:
        span_row=0
        next_token=spans.iloc[span_row]['span_start']
        to_print=text[:token_starts[next_token]]
        while span_row < spans.shape[0]:
            span_info=spans.iloc[span_row].to_dict()
            # start color:
            to_print+=f'\x1b[{colors[span_info["question id"]]}m'
            # add span text
            to_print+=text[token_starts[span_info['span_start']]:token_starts[span_info['span_end']+1]]
            # end color:
            to_print+=f'\x1b[0m'
            
            if span_row == spans.shape[0]-1:
                to_print+=text[token_starts[span_info['span_end']+1]:]
            else:
                next_span_start=spans.iloc[span_row+1]['span_start']
                if next_span_start > span_info['span_end']:
                    to_print+=text[token_starts[span_info['span_end']+1]:token_starts[next_span_start]]
                else:
                    to_print+='\n'
            span_row+=1
        print(to_print)

def get_colors():
    return [f'6;30;4{x}' for x in range(1,7)]

def print_legend(questions, colors=None):
    if colors is None:
        colors=get_colors()
    print('*'*100)
    print(47*' '+'LEGEND'+47*' ')
    print('*'*100)
    for i, question in enumerate(questions):
        print(f'\x1b[{colors[i]}m{question}\x1b[0m')
    print('*'*100)
    print('')
    
def find_spans(df, tokenizer, row_id, questions, all_tokens):
    spans=[]
    for i, question in enumerate(questions):
        for j, answer in enumerate(df.loc[row_id,question]):
            answer_tokens = tokenizer(answer, return_tensors='pt')['input_ids'][:,1:-1]
            results = find_sub_list(list(answer_tokens[0]), list(all_tokens[0]))
            if results is not None:
                span_start, span_end = results
                spans.append({'question id':i,
                              'answer id':j,
                              'span_start':span_start,
                              'span_end':span_end
                             })
    if len(spans)>0:       
        return pd.DataFrame(spans).sort_values(['span_start','span_end','question id'],axis=0,ascending=True)
    else:
        return pd.DataFrame(columns=['question id','answer id','span_start','span_end'])

def find_sub_list(sublist,lis,min_match_pct=0.5,allowable_mismatches=0):
    sublist_length = len(sublist)
    
    max_allowable_mismatches = int(sublist_length*(1-min_match_pct))
    if allowable_mismatches >= max_allowable_mismatches:
        print('ERROR: no matches found')
        return None
    
    for start in range(len(lis)-sublist_length):
        num_mismatches=0
        for j in range(sublist_length):
            if lis[start+j] != sublist[j]:
                num_mismatches+=1
                if num_mismatches > allowable_mismatches:
                    break
        if num_mismatches <= allowable_mismatches:
            return start, start + sublist_length - 1
        
    # if no match found, increase num_mismatches and try again
    return find_sub_list(sublist,lis,min_match_pct,allowable_mismatches+1)

def token_start_locations(text, tokenizer, tokens = None):
    if tokens is None:
        tokens = tokenizer(text, return_tensors='pt')['input_ids'][:,1:-1] # drop first and last tokens ([CLS] and [SEP])
    num_tokens = tokens.size()[1]
    
    token_starts=[]
    str_start=0
    for i in range(num_tokens):
        token_to_find=tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens[0,i:i+1]))
        find_index=text[str_start:].lower().find(token_to_find)
        str_start+=find_index
        token_starts.append(str_start)
    return token_starts

PyTorch version 1.6.0 available.
TensorFlow version 2.3.0 available.
loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/ktrapeznikov/albert-xlarge-v2-squad-v2/config.json from cache at C:\Users\dhilg/.cache\torch\transformers\c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f
Model config AlbertConfig {
  "architectures": [
    "AlbertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 16,
  "num_hidden_groups": 1,
  "num_hidden_laye

In [2]:
print_legend(questions)

****************************************************************************************************
                                               LEGEND                                               
****************************************************************************************************
Is there training provided?
Is there training already in place?
Has training been done?
Is training planned?
Is training in development?
What kind of training is provided?
****************************************************************************************************



In [3]:
# Note: the highlighting algorithm wasn't always displaying things properly. The following list of row_ids has been 
# validated such that the highlights are correct. Those documents with improper highlighting and those with no spans
# returned by the QA model have been filtered out. Only the first 50 rows were validated.
for row_id in [0,1,2,4,8,12,15,18,21,23,25,26,31,32,35,37,38,39,40,41,42,43,44,47,48,49]:
    print('*'*100)
    print(f'row_id = {row_id}')
    print_doc_with_highlights(df, tokenizer, row_id)

Token indices sequence length is longer than the specified maximum sequence length for this model (870 > 512). Running this sequence through the model will result in indexing errors


****************************************************************************************************
row_id = 0
****************************************************************************************************
                                               LEGEND                                               
****************************************************************************************************
Is there training provided?
Is there training already in place?
Has training been done?
Is training planned?
Is training in development?
What kind of training is provided?
****************************************************************************************************



Token indices sequence length is longer than the specified maximum sequence length for this model (1259 > 512). Running this sequence through the model will result in indexing errors


Modern Slavery Statement

Ua

> Responsibility > Modern Slavery Statement
Anti-Slavery and Human Trafficking Statement for the year ending 31 March 2018
Introduction from the Board
AO World plc (“AO”) fully supports the government’s objectives to eradicate modern slavery and human trafficking.
About AO World plc
We sell major and small domestic appliances and consumer electronics in the UK, Germany and the Netherlands and deliver them via our in-house logistics business and carefully selected third parties. We also provide ancillary services such as the installation of new and collection of old products and offer product protection plans and customer finance. Via our state of the art facility we are also able to carry out the recycling of waste appliances in the UK. We have a unique and vibrant culture and a team of people who genuinely care more about our business and its customers and currently employ over 2,800 people. For further details on the structure of our business and our Cor

Token indices sequence length is longer than the specified maximum sequence length for this model (1690 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (1539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1995 > 512). Running this sequence through the model will result in indexing errors


6/5/2019
(/)

Modern Slavery Act 2015 | Close Brothers Group plc



Modern Slavery Act 2015
Close Brothers Group anti-slavery and human trafficking statement for the financial year ended 31 July 2018
Group overview

https://www.closebrothers.com/modern-slavery-act-2015

1/7

6/5/2019

Modern Slavery Act 2015 | Close Brothers Group plc
Close Brothers Group plc (“the group” or “Close Brothers”) is a leading merchant banking group providing lending, deposit taking, wealth management services and securities trading.

Close Brothers was established in 1878 and today employs around 3,300 people, principally in the UK. Close Brothers Group plc is listed on the London Stock Exchange and is a member of the FTSE 250.

The group is comprised of three divisions – Banking, Securities and Asset Management, the Banking division being the largest and contributing over 90% of the group’s profits.

Close Brothers’ Banking division, comprising Close Brothers Limited and its subsidiaries, provides spec

Token indices sequence length is longer than the specified maximum sequence length for this model (745 > 512). Running this sequence through the model will result in indexing errors


MODERN SLAVERY STATEMENT 2018
OUR BUSINESS
CHANEL was established in Paris 1909 by Mademoiselle Chanel as a luxury brand selling luxury fashion. CHANEL is an independent company that creates, markets, distributes and sells luxury Fragrances & Beauty, Fashion (including Haute Couture, Accessories and Eyewear) and Watches & Fine Jewellery products. Today, the house of CHANEL operates across 70 countries worldwide.
CHANEL is dedicated to creating innovative and desirable products of the highest quality and finest craftsmanship. We invest in the excellence of our products and favour a long-term vision to ensure the longevity of the brand. A large proportion of our products are manufactured at in-house facilities in France as well as with select third party partners. Our supply base consists of more than 3000 suppliers around the world, split between what we term "direct procurement" (e.g. finished goods, componentry and raw materials) and "indirect procurement" (e.g. sales, promotional and

CALIFORNIA TRANSPARENCY IN SUPPLY CHAINS ACT DISCLOSURE
Genesco and its subsidiaries (collectively, "Genesco" or the "Company") sell consumer goods through both wholesale and retail channels. Its retail operations sell both merchandise manufactured to the Company’s order by unrelated contract manufacturers and bearing the Company’s owned or licensed brands, and merchandise manufactured for and purchased from unrelated wholesale distributors and bearing their brands.
Ethical Sourcing Policy
Genesco is committed to principles of ethical business practice and recognition of the dignity of others, including responsible labor practices. The same policies relating to slavery and forced labor apply across all of Genesco’s operations. We therefore discuss in this Statement our efforts to eradicate slavery and forced labor from our supply chains for goods offered for sale on an enterprise-wide basis. However, some of Genesco’s subsidiaries are not subject to the California Transparency in Suppl